### RNN Assignment

import needed libraries

In [141]:
import numpy as np
import pandas as pd
from keras.utils.np_utils import to_categorical
from keras.layers import GRU, Dropout
from keras.models import Sequential
from keras.layers import LSTM, SimpleRNN, Dense
import keras

import data

get max length

In [142]:
max_len=0
all_data_x=[]
all_data_y=[]
for subject_id in range(1, 11):
    file_name = f"mHealth_subject{subject_id}.log"
    
    with open(file_name, "r", encoding="utf8") as file:
        count={}
        prev_label=0
        data=[]
        for line in file:
            tokens = line.split()
            label = int(tokens[-1])
            if(label!= 0):
                if(prev_label!=label and prev_label!=0):
                    all_data_x.append(data)
                    all_data_y.append(prev_label)
                    count[prev_label]=len(data)
                           
                columns=len(tokens)-1
                if(prev_label !=label):
                    data+=[tokens[:-1]]
                prev_label=label
            
        # print(count)
        all_data_x.append(data)
        all_data_y.append(prev_label)
        count[prev_label]=len(data)
        for value in count.values():
            if(value>max_len):
                max_len=value
                

print the maximum length

In [143]:
print("maximun lenth of data =",max_len)

maximun lenth of data = 12


In [144]:
max_column=23 ## using question
num_labels=12
data_count=10*num_labels

In [145]:
def get_em_data(data,max_len,max_column):
    embedding_data= np.zeros((len(data), max_len, max_column))
    
    for index in range(len(data)):
        num_times=len(data[index])
        for i in range(num_times):
            embedding_data[index][i]= np.array(data[index][i], dtype=float)
    return embedding_data

Convert into numpy array

In [146]:
y_array=np.array(all_data_y)

print a sample array

In [147]:
y_array

array([ 1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 12,  5,  5,  1,  2,  3,  4,
        6,  7,  8, 10, 11, 12,  9,  1,  2,  3,  4,  6,  7,  8,  9, 10, 11,
       12,  5,  5,  1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 12,  1,  2,  3,
        4,  6,  7,  8,  9, 10, 11, 12,  5,  1,  2,  3,  4,  6,  7,  8,  9,
       10, 11, 12,  5,  5,  1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 12,  1,
        3,  2,  4,  5,  6,  7,  8,  9, 10, 11, 12,  1,  2,  3,  4,  6,  7,
        8,  9, 10, 11, 12,  5,  5,  1,  2,  3,  4,  6,  7,  8,  9, 10, 11,
       12])

In [148]:
x_array=get_em_data(all_data_x,max_len +1,max_column)

convert to categorical

In [149]:
y_convert =y_array-1
y_array_c = to_categorical(y_convert)


create a model

In [150]:
model = Sequential()

In [151]:
print(max_column)

23


This is the model LSTM

In [152]:
model.add(LSTM(units= 256 ,return_sequences=True,input_shape=(max_len+1,max_column)))
model.add(Dropout(0.25))
model.add(LSTM(units= 512,return_sequences=True))
model.add(Dropout(0.25))
model.add(LSTM(units= 64))
model.add(Dropout(0.25))
model.add(Dense(units=128,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(units=64,activation='relu'))
model.add(Dense(units=num_labels,activation='softmax'))

###SimpleRNN
###Bidirectional(GRU)
#model.add(GRU(units= 128,return_sequences=True)

Define losss function

In [153]:
model.compile(optimizer='adam',loss=keras.losses.categorical_crossentropy,metrics=['accuracy'])

Fit the model

In [155]:
result=model.fit(x_array,y_array_c,epochs=20,batch_size=20,validation_split=0.2, shuffle=True)
#batch_size=20,validation_split=0.2

Epoch 1/50
5/5 [==============================] - 1s 204ms/step - loss: 2.4934 - accuracy: 0.0729 - val_loss: 2.4855 - val_accuracy: 0.0833
Epoch 2/50
5/5 [==============================] - 1s 229ms/step - loss: 2.4818 - accuracy: 0.0521 - val_loss: 2.4856 - val_accuracy: 0.0833
Epoch 3/50
5/5 [==============================] - 1s 207ms/step - loss: 2.4884 - accuracy: 0.1042 - val_loss: 2.4853 - val_accuracy: 0.0833
Epoch 4/50
5/5 [==============================] - 1s 226ms/step - loss: 2.4999 - accuracy: 0.0625 - val_loss: 2.4853 - val_accuracy: 0.0833
Epoch 5/50
5/5 [==============================] - 1s 199ms/step - loss: 2.4832 - accuracy: 0.1250 - val_loss: 2.4854 - val_accuracy: 0.0833
Epoch 6/50
5/5 [==============================] - 1s 207ms/step - loss: 2.4929 - accuracy: 0.0625 - val_loss: 2.4855 - val_accuracy: 0.0833
Epoch 7/50
5/5 [==============================] - 1s 182ms/step - loss: 2.4917 - accuracy: 0.1042 - val_loss: 2.4853 - val_accuracy: 0.0833
Epoch 8/50
5/5 [====

Predict some values

In [164]:
predicted = model.predict(x_array[2:8])

1/1 [==============================] - 0s 83ms/step


In [165]:
predicted[0]

array([0.08792985, 0.0859952 , 0.08033592, 0.08250325, 0.08043326,
       0.08550989, 0.08530441, 0.08192568, 0.08191549, 0.08557224,
       0.08124387, 0.08133093], dtype=float32)

In [166]:
predicted_labels = np.argmax(predicted, axis=1)

Find the accuracy

In [167]:
training_accuracy = result.history['accuracy']
validation_accuracy = result.history['val_accuracy']


In [168]:
print( "final training accuracy = ",training_accuracy[-1])
print( "final testing accuracy = ",validation_accuracy[-1])

final training accuracy =  0.0729166641831398
final testing accuracy =  0.0833333358168602


In [169]:
print( "maximum training accuracy = ",max(training_accuracy))
print( "maximum testing accuracy = ",max(validation_accuracy))

maximum training accuracy =  0.1354166716337204
maximum testing accuracy =  0.0833333358168602


Save the model

In [170]:
model.save("LSTM_model_3.h5")